In [ ]:
pip install pytorch-tabnet

In [ ]:
pip install shap

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import learning_curve
import shap

#### Upload data

In [2]:
df = pd.read_csv('openfoodfacts_cleaned.csv')

/var/folders/_p/d7rhktfx23s7f0t994bd2j500000gn/T/ipykernel_37406/2758970970.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('openfoodfacts_cleaned.csv')


### Features Choosing Manually

In [ ]:
food_countries_categories = df['countries_en'].unique()
# The output is: Number of unique categories in 'countries_en': 5529

# Select the top 5 countries with the largest sample size to focus on the most prominent patterns
# This helps in capturing the main features of countries and reduces the influence of noise
selected_countries = ['France', 'United States', 'Italy', 'Spain', 'Germany']
df = df[df['countries_en'].isin(selected_countries)]

In [ ]:
# remain columns I want to explore further
columns_needed = [
    'countries_en', 'food_groups_en', 'nutriscore_score','energy-kcal_100g', 'saturated-fat_100g', 
    'sugars_100g', 'sodium_100g', 'proteins_100g', 'fiber_100g', 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

# extract coulumns 
df = df[columns_needed]

In [ ]:
# Rename columns to reuse them briefly
df = df.rename(columns={
    'fruits-vegetables-nuts-estimate-from-ingredients_100g': 'FVN_estimate',
    'saturated-fat_100g': 'sat_fat',
    'energy-kcal_100g': 'energy_kcal',
    'sugars_100g': 'sugar',
    'sodium_100g': 'sodium',
    'proteins_100g': 'protein',
    'fiber_100g': 'fiber',
    'countries_en': 'countries',
    'food_groups_en': 'food_groups',
    'nutriscore_score': 'nutriscore'  
})

# Output the updated column names to check
print (df.columns)

### Missing Values Processing

In [ ]:
# Check for missing values in each column
missing_values = df.isnull().sum()

In [ ]:
# Filter out rows where 'nutrition-score-fr_100g' (Nutri-score) column has missing values
df = df[df['nutriscore'].notna()]

In [ ]:
# Remove rows with missing values in 'fiber_100g' and 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
df_cleaned = df.dropna(subset=['fiber', 'FVN_estimate'])

In [ ]:
# Remove rows where 'food_groups_en' column has missing values
df_cleaned = df_cleaned.dropna(subset=['food_groups'])

print(f"Number of rows and columns in the dataset: {df.shape}")

### Outliers Processing

#### 'nutriscore_score'

In [ ]:
# Check the range of Nutri-Score
min_value = df['nutriscore'].min()
max_value = df['nutriscore'].max()

print(f"Range of nutriscore_score: {min_value} to {max_value}")

#### 'Food_groups_en' 

In [ ]:
# Remove rows where 'food_groups_en' is 'Unknown' or 'Alcoholic beverages'
# These categories have too few samples to have a significant impact on the analysis
df = df[~df['food_groups'].isin(['Unknown', 'Alcoholic beverages'])]

#### energy-kcal_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'energy-kcal_100g'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['energy_kcal'])
plt.title('Boxplot of energy_kcal')
plt.xlabel('energy_kcal')
plt.show()

In [ ]:
# Remove rows where the 'energy-kcal_100g' column has values greater than 1000
df = df[df['energy_kcal'] <= 1000]

#### saturated-fat_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'saturated-fat_100g'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['sat_fat'])
plt.title('Boxplot of sat_fat')
plt.xlabel('sat_fat')
plt.show()

In [ ]:
# Remove rows where the 'saturated-fat_100g' column has values greater than 1000
df = df[df['sat_fat'] <= 100]

#### sugars_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'sugars_100g'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['sugar'])
plt.title('Boxplot of sugar')
plt.xlabel('sugar')
plt.show()

In [ ]:
# Remove rows where the 'sugars_100g' column has values greater than 100
df = df[df['sugar'] <= 100]
# Print the shape of the filtered dataset to confirm the number of rows and columns
print(f"Filtered dataset shape: {df.shape}")

#### sodium_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'sodium_100g'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_cleaned['sodium'])
plt.title('Boxplot of sodium')
plt.xlabel('sodium')
plt.show()

In [ ]:
# List of food categories where higher sodium content (>3g/100g) is considered reasonable
valid_categories = [
    'Fats and sauces,Dressings and sauces',
    'Composite foods,One-dish meals',
    'Salty snacks, Salty and fatty products',
    'Milk and dairy products, Cheese'
]

# Remove rows where 'sodium_100g' > 3g/100g that do not belong to valid categories
df = df[~((df['sodium'] > 3) & (df['food_groups'].apply(lambda x: x not in valid_categories)))]

# Print the shape of the filtered dataset to confirm the number of rows and columns
print(f"Filtered dataset shape: {df.shape}")

#### proteins_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'proteins_100g‘
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['protein'])
plt.title('Boxplot of protein')
plt.xlabel('protein')
plt.show()

In [ ]:
# Remove rows where the 'sugars_100g' column has values greater than 100
df = df[df['protein'] <= 50]

#### fiber_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'fiber_100g‘
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['fiber'])
plt.title('Boxplot of fiber')
plt.xlabel('fiber')
plt.show()

In [ ]:
# Remove rows where the 'sugars_100g' column has values greater than 100
df = df[df['fiber'] < 50]

#### fruits-vegetables-nuts-estimate-from-ingredients_100g

In [ ]:
# Create a boxplot to visualize the distribution of 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['FVN_estimate'])
plt.title('Boxplot of FVN_estimate')
plt.xlabel('FVN_estimate')
plt.show()

In [ ]:
# Remove rows where the 'fruits-vegetables-nuts-estimate-from-ingredients_100g' column has values greater than 100

df = df[df['FVN_estimate'] <= 100]
df = df[df['FVN_estimate'] >= 0]

# Print the shape of the filtered dataset to confirm the number of rows and columns
print(f"Filtered dataset shape: {df.shape}")

### Aggregate the food groups into broader, fewer categories （part of feature engineering）

In [ ]:
# Create a new column by extracting the part before the last comma of 'food_groups_en'  
df['food_groups_new'] = df['food_groups'].str.rsplit(',', n=1).str[0]

print(df_cleaned[['food_groups', 'food_groups_new']].head())

# Replace specified categories with 'Fish, Meat, Eggs'
df['food_groups_new'] = df['food_groups_new'].replace(
    ['Fish‚ Meat‚ Eggs,Fish and seafood', 'Fish‚ Meat‚ Eggs', 'Fish‚ Meat‚ Eggs,Meat'],
    'Fish, Meat, Eggs')

# Check the unique values after the replacement
print(df['food_groups_new'].unique())


In [ ]:
#deal with specific categories
df['food_groups_new'] = df['food_groups_new'].replace(
    ['Fish‚ Meat‚ Eggs,Fish and seafood', 'Fish‚ Meat‚ Eggs', 'Fish‚ Meat‚ Eggs,Meat'],
    'Fish, Meat, Eggs')

# Check the results after processing
print(df['food_groups_new'].unique())

In [ ]:
df = df.drop(columns=['food_groups'])

df.head()

#### Data Split

In [ ]:
# Split the DataFrame into 85% training and tuning set, 15% final Test set
train_set, final_test_set = train_test_split(df, test_size=0.15, random_state=42)

#### import csv file

In [ ]:
# Export the training set to a CSV file
train_set.to_csv('train_set.csv', index=False)  

# Export the final Test set to a CSV file
final_test_set.to_csv('final_test_set.csv', index=False)  

## Feature Engineering

In [ ]:
# import  train data
train_df = pd.read_csv('/content/drive/MyDrive/laylatest/train_set.csv')

train_df.shape

In [ ]:
# import test data 
test_df = pd.read_csv('/content/drive/MyDrive/laylatest/final_test_set.csv')

test_df.shape

### Encode the train dataset

#### Encoding 'countries' by Label encoder

In [ ]:
# Initialize LabelEncoder
country_label_encoder = LabelEncoder()

# Fit and transform the 'countries_en' column in the dataset
train_df['countries_encoded'] = country_label_encoder.fit_transform(train_df['countries'])

# Verify the encoding
print(train_df[['countries', 'countries_encoded']].head())

#### Encoding ‘food_groups’ by Label encoding

In [ ]:
# Initialize LabelEncoder
food_label_encoder = LabelEncoder()

# Fit and transform the 'food_groups_en' column in the dataset
train_df['food_groups_encoded'] = food_label_encoder.fit_transform(train_df['food_groups_new'])

# Verify the encoding
print(train_df[['food_groups_new', 'food_groups_encoded']].head())

#####  Standardize column names and remove unnecessary features

In [ ]:

# Replace special characters in column names with '_'
train_df.columns = [re.sub('[^A-Za-z0-9_]+', '_', col) for col in train_df.columns]

train_df.drop(columns=['countries', 'food_groups_new'], inplace=True)

### Encode the final test dataset.

#### Encoding 'countries' by Label encoder

In [ ]:
# Initialize LabelEncoder
country_label_encoder = LabelEncoder()

# Fit and transform the 'countries_en' column in the dataset
test_df['countries_encoded'] = country_label_encoder.fit_transform(test_df['countries'])

# Verify the encoding
print(test_df[['countries', 'countries_encoded']].head())

#### Encoding ‘food_groups’ by Label encoding

In [ ]:
# Initialize LabelEncoder
food_label_encoder = LabelEncoder()

# Fit and transform the 'food_groups_en' column in the dataset
test_df['food_groups_encoded'] = food_label_encoder.fit_transform(test_df['food_groups_new'])

# Verify the encoding
print(test_df[['food_groups_new', 'food_groups_encoded']].head())

#####  Standardize column names and remove unnecessary features

In [ ]:
# Replace special characters in column names with '_'
test_df.columns = [re.sub('[^A-Za-z0-9_]+', '_', col) for col in test_df.columns]
# Remove unnecessary columns and remain encoded columns 
test_df.drop(columns=['countries', 'food_groups_new'], inplace=True)

## Model Training

#### Set the test dataset aside and only use it to test the final model performance after hyperparameter tuning

In [ ]:
# Split test_df into features (X) and target (y)
X_test = test_df.drop(columns=['nutriscore'])
y_test = test_df['nutriscore']

### Data split and Mean Baseline Model

In [ ]:
# Split train_df into features (X) and target (y)
X = train_df.drop(columns=['nutriscore'])
y = train_df['nutriscore']

In [ ]:
# Calculate the mean of the target variable from the training data
mean_baseline = np.mean(y)
print(f'Mean Baseline Value (calculated from training data): {mean_baseline}')

# Create baseline predictions for the test data (mean of training data)
baseline_predictions = np.full(len(y_test), mean_baseline)  # Use the mean of y_train to predict all test data

# Calculate evaluation metrics on the validation data
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
mse = mean_squared_error(y_test, baseline_predictions)
r2_baseline = r2_score(y_test, baseline_predictions)
rmse = np.sqrt(mse)


print(f'MAE for Baseline Model: {mae_baseline}')
print(f'MSE for the Baseline Model: {mse}')
print(f'R² for Baseline Model: {r2_baseline}')
print(f'RMSE for Baseline Model: {rmse}')

### XGBoost 

#### Baseline Model (Before Hyperparameter Tuning)

In [ ]:
# Initialize the XGBoost Regressor with default parameters
xgb_model = xgb.XGBRegressor(random_state=42)

# Perform 5-fold cross-validation with MAE as the metric
cv_mae_scores = -cross_val_score(xgb_model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)

mean_mae = cv_mae_scores.mean()
print(f'MAE for the baseline XGBoost model:{mean_mae}')

#### Determine appropriate hyperparameter ranges --- Learning curve

##### n_estimators

In [ ]:
# Define the range of n_estimators to explore its effect on model performance
n_estimators_range = [50, 100, 150, 200, 250, 300, 400]

# Initialize lists to store training and test errors for each n_estimators value
train_mae_scores = []
val_mae_scores = []

for n in n_estimators_range:
    # Create an XGBoost model with the current n_estimators and fixed max_depth
    model = xgb.XGBRegressor(n_estimators=n, max_depth=6, learning_rate=0.1, random_state =42)

    # Calculate MAE on validation dataset of every k-fold and append to the list (as MAE on test data)
    val_mae = -cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
    val_mae_scores.append(val_mae.mean())

    # Calculate training error and append to the list
    model.fit(X, y)
    train_mae = mean_absolute_error(y, model.predict(X))
    train_mae_scores.append(train_mae)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, train_mae_scores, label="Training MAE", color="blue")
plt.plot(n_estimators_range, val_mae_scores, label="Validation MAE", color="orange")
plt.xlabel("n_estimators")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Learning Curve for n_estimators")
plt.legend()
plt.grid()
plt.show()

##### max_depth

In [ ]:
# Repeat the steps above for max_depth to explore its effect on model performance
train_mae_scores = []
val_mae_scores = []

# Define the range of max_depth to explore its impact
max_depth_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for depth in max_depth_range:
    # Create an XGBoost model with the current max_depth and fixed n_estimators
    model = xgb.XGBRegressor(n_estimators=150, max_depth=depth, learning_rate=0.1, random_state =42)

    # Use cross-validation to obtain validation MAE
    val_mae = -cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
    val_mae_scores.append(val_mae.mean())

    # Calculate MAE on training data
    model.fit(X, y)
    train_mae = mean_absolute_error(y, model.predict(X))
    train_mae_scores.append(train_mae)

# Plot learning curve for max_depth
plt.figure(figsize=(10, 6))
plt.plot(max_depth_range, train_mae_scores, label="Training MAE", color="blue")
plt.plot(max_depth_range, val_mae_scores, label="Validation MAE", color="orange")
plt.xlabel("n_estimators")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Learning Curve for max_depth")
plt.legend()
plt.grid()
plt.show()

#### Hyperparameter Optimization by Grid Search

In [ ]:
# Define the parameter grid to search （19min 47s)
param_grid = {
    'n_estimators': [150, 200, 250],          # Number of boosting rounds (trees)
    'max_depth': [ 5, 6, 7],                 # Maximum depth of each tree
    'learning_rate': [0.01, 0.05, 0.1],           # Learning rate
    'min_child_weight': [1, 2, 3]
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  # Scoring metric
    cv=5,                               # 5-fold cross-validation
    verbose=1,                          # Display progress
    n_jobs=-1                           # Use all available cores
)

# Perform the grid search
grid_search.fit(X, y)

# Print the best parameters found by GridSearchCV
best_params_xgboost = grid_search.best_params_
print(f"Best parameters: {best_params_xgboost}")

# Get the best estimator (model with best hyperparameters)
best_xgb_model = grid_search.best_estimator_

#### Model Performance on all Train data

In [ ]:
# Re-initialize the model with the best parameters and fit it on the entire training set
best_xgb_model = xgb.XGBRegressor(**best_params_xgboost, random_state=42)
best_xgb_model.fit(X, y)

# Make predictions on the training set
y_train_pred = best_xgb_model.predict(X)

# Evaluate the performance on the training set
train_mae = mean_absolute_error(y, y_train_pred)
train_mse = mean_squared_error(y, y_train_pred)
train_r2 = r2_score(y, y_train_pred)
train_rmse = np.sqrt(train_mse)

# Print the results for the training set
print(f"MAE on the training set: {train_mae}")
print(f"MSE on the training set: {train_mse}")
print(f"RMSE on the training set: {train_rmse}")
print(f"R² on the training set: {train_r2}")

#### XGBoost performance on test data with best hyperparameters

In [ ]:
# Make predictions on the test set
y_pred = best_xgb_model.predict(X_test)

# Evaluate the performance of the model on the test set
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

# Print the results
print(f"MAE for the best XGBoost model: {mae}")
print(f"MSE on the final test data: {mse}")
print(f"RMSE on the training set: {rmse}")
print(f"R² on the final test data: {r2}")

### LightGBM

#### Baseline Model (Before Hyperparameter Tuning)

In [ ]:
# Initialize the LightGBM Regressor with default parameters
lgb_model = lgb.LGBMRegressor(random_state=42)

# Perform 5-fold cross-validation with MAE as the metric
cv_mae_scores = -cross_val_score(lgb_model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)

mean_mae = cv_mae_scores.mean()
print(f'MAE for the baseline LightGBM model:{mean_mae}')

#### Determine appropriate hyperparameter ranges --- Learning curve

##### n_estimators

In [ ]:
# Define the range of n_estimators to explore
n_estimators_range = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

# Initialize lists to store training and validation errors for each n_estimators value
train_mae_scores = []
val_mae_scores = []

for n in n_estimators_range:
    # Create a LightGBM model with the current n_estimators
    model = lgb.LGBMRegressor(n_estimators=n, max_depth=6, learning_rate=0.1, num_leaves=30, random_state=42)

    # Calculate MAE on validation dataset of every k-fold and append to the list (as MAE on validation data)
    val_mae = -cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
    val_mae_scores.append(val_mae.mean())

    # Fit model on entire training data and calculate MAE for training set
    model.fit(X, y)
    train_mae = mean_absolute_error(y, model.predict(X))
    train_mae_scores.append(train_mae)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, train_mae_scores, label="Training MAE", color="blue")
plt.plot(n_estimators_range, val_mae_scores, label="Validation MAE", color="orange")
plt.xlabel("n_estimators")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Learning Curve for LightGBM n_estimators")
plt.legend()
plt.grid()
plt.show()

##### max_depth

In [ ]:
# Repeat the steps above for max_depth to explore its effect on model performance
train_mae_scores = []
val_mae_scores = []

# Define the range of max_depth to explore its impact
max_depth_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for depth in max_depth_range:
    # Create an LightGBM model with the current max_depth and fixed n_estimators
    model = lgb.LGBMRegressor (n_estimators=150, max_depth=depth, learning_rate=0.1, num_leaves=30, random_state =42)

    # Use cross-validation to obtain validation MAE
    val_mae = -cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
    val_mae_scores.append(val_mae.mean())

    # Calculate MAE on training data
    model.fit(X, y)
    train_mae = mean_absolute_error(y, model.predict(X))
    train_mae_scores.append(train_mae)

# Plot learning curve for max_depth
plt.figure(figsize=(10, 6))
plt.plot(max_depth_range, train_mae_scores, label="Training MAE", color="blue")
plt.plot(max_depth_range, val_mae_scores, label="Validation MAE", color="orange")
plt.xlabel("n_estimators")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Learning Curve for max_depth")
plt.legend()
plt.grid()
plt.show()

##### num_leaves

In [ ]:
# Define the range of num_leaves to explore
num_leaves_range = [20, 30, 40, 50, 60]

# Initialize lists to store training and validation errors for each num_leaves value
train_mae_scores = []
val_mae_scores = []

for num_leaves in num_leaves_range:
    # Create a LightGBM model with the current num_leaves
    model = lgb.LGBMRegressor(num_leaves=num_leaves, n_estimators=150, max_depth=6, learning_rate=0.1, random_state=42)

    # Calculate MAE on validation dataset of every k-fold and append to the list (as MAE on validation data)
    val_mae = -cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
    val_mae_scores.append(val_mae.mean())

    # Fit model on entire training data and calculate MAE for training set
    model.fit(X, y)
    train_mae = mean_absolute_error(y, model.predict(X))
    train_mae_scores.append(train_mae)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(num_leaves_range, train_mae_scores, label="Training MAE", color="blue")
plt.plot(num_leaves_range, val_mae_scores, label="Validation MAE", color="orange")
plt.xlabel("num_leaves")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("Learning Curve for LightGBM num_leaves")
plt.legend()
plt.grid()
plt.show()

#### Hyperparameter optimization by Grid Search

In [ ]:
# Define the parameter grid to search (20min 14s)
param_grid = {
    'n_estimators': [150, 200, 250],              # Number of boosting rounds (trees)
    'max_depth': [5, 6, 7, 8],                  # Maximum depth of each tree
    'learning_rate': [0.01, 0.05, 0.1],            # Learning rate
    'num_leaves': [30, 40, 50]                 # Number of leaves in one tree
    # 'min_child_samples': [10, 20, 30],           # Minimum number of samples in a child (leaf)
}


# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=lgb_model,
    param_grid=param_grid,
    cv=5,                         # 5-fold cross-validation
    scoring='neg_mean_absolute_error',  # Evaluation metric
    verbose=1,                    # Display progress
    n_jobs=-1                     # Use all CPU cores
)

# Perform the grid search
grid_search.fit(X, y)

# Get the best parameters found by GridSearchCV
best_params_LightGBM = grid_search.best_params_
print(f"Best parameters: {best_params_LightGBM}")

# Get the best estimator (model with the best hyperparameters)
best_lgb_model = grid_search.best_estimator_

#### Model Performance on all Train data

In [ ]:
# Re-initialize the model with the best parameters and fit it on the entire training set
best_lgb_model = lgb.LGBMRegressor(**best_params_LightGBM, random_state=42)
best_lgb_model.fit(X, y)

# Make predictions on the training set
y_train_pred = best_lgb_model.predict(X)

# Evaluate the performance on the training set
train_mae = mean_absolute_error(y, y_train_pred)
train_mse = mean_squared_error(y, y_train_pred)
train_r2 = r2_score(y, y_train_pred)
train_rmse = np.sqrt(train_mse)

# Print the results for the training set
print(f"MAE on the training set: {train_mae}")
print(f"MSE on the training set: {train_mse}")
print(f"RMSE on the training set: {train_rmse}")
print(f"R² on the training set: {train_r2}")

#### LightGBM performance on test data with best hyperparameters

In [ ]:
# Make predictions on the test set
y_pred = best_lgb_model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

# Print the results
print(f"MAE for the best LightGBM model: {mae}")
print(f"MSE for the best LightGBM model: {mse}")
print(f"R² for the best LightGBM model: {r2}")
print(f"RMSE for the best LightGBM model: {rmse}")

### TabNet

In [ ]:
# Convert training and validation data to NumPy arrays (TabNet requires NumPy arrays)
X_np = X.values
X_test_np = X_test.values

# Reshape y_train and y_val to 2D arrays (TabNet expects 2D targets)
y_np = y.values.reshape(-1, 1)  # Reshape y_train
y_test_np = y_test.values.reshape(-1, 1)      # Reshape y_val

#### Baseline Model

In [ ]:
# Initialize TabNet with default parameters (1h+)
tabnet_model = TabNetRegressor(device_name='cuda',verbose=1, seed=42)


# Perform 5-fold cross-validation with MAE as the metric
cv_mae_scores = -cross_val_score(tabnet_model, X_np, y_np, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)

# Calculate the mean MAE from the cross-validation
mean_mae = cv_mae_scores.mean()
print(f'MAE for the baseline TabNet model: {mean_mae}')

#### Hyperparameter optimization by Grid Search

In [ ]:
# Define the parameter grid to search
param_grid = {
    'n_d': [8, 16, 24, 32],               # Dimensionality of the decision prediction layer
    'n_a': [8, 16, 24, 32],               # Dimensionality of the attention embedding for each mask
    'n_steps': [3, 5, 7],             # Number of steps in the architecture
    'gamma': [1.0, 1.5],         # Scaling factor for the attention updates
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=tabnet_model,
    param_grid=param_grid,
    cv=5,                         # 5-fold cross-validation
    scoring='neg_mean_absolute_error',  # Evaluation metric
    verbose=1,                    # Display progress
    n_jobs=-1                     # Use all CPU cores
)

# Perform the grid search
grid_search.fit(X_np, y_np)

# Get the best parameters found by GridSearchCV
best_params_tabnet = grid_search.best_params_
print(f"Best parameters: {best_params_tabnet}")

# Get the best estimator (model with the best hyperparameters)
best_tabnet_model = grid_search.best_estimator_

#### Model Performance on all Train data

In [ ]:
# Re-initialize the model with the best parameters and fit it on the entire training set
best_tabnet_model = TabNetRegressor(**best_params_tabnet, random_state=42)
best_tabnet_model.fit(X_np, y_np)

# Make predictions on the training set
y_train_pred = best_tabnet_model.predict(X_np)

# Evaluate the performance on the training set
train_mae = mean_absolute_error(y_np, y_train_pred)
train_mse = mean_squared_error(y_np, y_train_pred)
train_r2 = r2_score(y_np, y_train_pred)
train_rmse = np.sqrt(train_mse)

# Print the results for the training set
print(f"MAE on the training set: {train_mae}")
print(f"MSE on the training set: {train_mse}")
print(f"RMSE on the training set: {train_rmse}")
print(f"R² on the training set: {train_r2}")

#### TabNet performance on test data with best hyperparameters

In [ ]:
# Make predictions on the validation set
y_pred = best_tabnet_model.predict(X_test_np)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_np, y_pred)
mse = mean_squared_error(y_test_np, y_pred)
r2 = r2_score(y_test_np, y_pred)
rmse = np.sqrt(mse)

# Print the results
print(f"MAE for the best LightGBM model: {mae}")
print(f"MSE for the best LightGBM model: {mse}")
print(f"R² for the best LightGBM model: {r2}")
print(f"RMSE for the best LightGBM model: {rmse}")

### SHAP on best-performing model (XGBoost)

#### Global Feature Importance

In [ ]:
# Define the best XGBoost model with the optimal parameters
best_xgb_model = xgb.XGBRegressor(
    learning_rate=0.1,
    max_depth=8,
    min_child_weight=1,
    n_estimators=250,
    random_state=42
)
# Fit the model on the training data
best_xgb_model.fit(X, y)

# Create a SHAP explainer for the XGBoost model
explainer = shap.TreeExplainer(best_xgb_model)

# Calculate SHAP values using the explainer
shap_values = explainer(X_test)

In [ ]:
# Plot SHAP global feature importance as a bar chart
shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig("shap_global_feature_importance_barplot.png")
plt.clf()

# Plot SHAP feature impact as a dot plot
shap.summary_plot(shap_values, X_test, show=False)
plt.tight_layout()
plt.savefig("shap_global_feature_importance_dotplot.png")
plt.clf()

#### SHAP Dependence Plot

In [ ]:
# Ensure SHAP values are in array format
shap_values_array = shap_values.values if hasattr(shap_values, 'values') else shap_values

# List of features for SHAP dependence plots
feature_list = ["countries", "food_groups", "sat_fat", "sugar", "sodium", "fiber", "energy_kcal", "FVN_estimate", "protein"]

# Loop through each feature to create and save SHAP dependence plots
for feature in feature_list:
    shap.dependence_plot(feature, shap_values_array, X_test, show=False)
    plt.title(f"SHAP Dependence Plot for {feature}")
    plt.ylabel(f"SHAP Value for {feature}")
    plt.tight_layout()
    plt.savefig(f"shap_dependence_plot_{feature}.png")
    plt.clf()